**18_MAW_iii**

In [1]:
import os ; import pathlib as pl ; import flopy ; import matplotlib.pyplot as plt ; import numpy as np; from pathlib import Path  
from flopy.plot.styles import styles ; import shapefile as shp ; import matplotlib as mpl ; import pandas as pd  

In [2]:
sim_ws    = Path("./18_MAW_iii")                    ; sim_ws.mkdir(exist_ok=True) ; sim_name = "ex-gwf-maw-p03"
exe_name  = 'E:\\15_REPOS\\00_BETAMI\\bin\\mf6.exe' ; workspace = figs_path = sim_ws ; gwf_name = sim_name ; output_folder = sim_ws   

In [3]:
#parameters = {"ex-gwf-maw-p03b": {"simulation": "multi-aquifer well"},  "ex-gwf-maw-p03c": {"simulation": "high conductivity"}}
name = 'mult'    
nper = 1 ;  nlay_r = 21 ;nrow_r = 1 ;ncol_r = 200   ; maw_highK = 1000000000 ;nouter = 500 ;ninner = 100 ;hclose = 0.000000001 ;rclose = 0.0001
strt = 10 ; nlay = 41   ;nrow = 16  ;ncol = 27 ;delr_r = 50 ;delc_r = 1 ;top = 10 ;aq_bottom = -205 ; k11 = 250 ;k33 = 50 ;recharge = 0.004566 
H1 = 0 ;maw_loc = (15,13) ;maw_lay = (1,12) ;maw_radius = 0.1333 ;maw_bot = -65   ; length_units = "feet" ; time_units = "days" 

delr = [10.0 ,10.0 ,9.002 ,6.0 ,4.0 ,3.0 ,2.0 ,1.33 ,1.25 ,1.00 ,1.00 ,0.75 ,0.50 ,0.333 ,0.50 ,0.75 ,1.00 ,1.00 ,1.25 ,1.333 ,
        2.0 ,3.0 ,4.0 ,6.0 ,9.002 ,10.0 ,10.0]      ; delc = [10, 9.38, 9, 6, 4, 3, 2, 1.33, 1.25, 1, 1, 0.75, 0.5, 0.3735, 0.25, 0.1665]
tdis_ds = ((1.0, 1, 1.0),) ; extents = (0.0, np.array(delr).sum(), 0.0, np.array(delc).sum())         ;shape2d=(nrow,ncol);shape3d=(nlay,nrow,ncol)
nconn = 2 + 3 * (maw_lay[1] - maw_lay[0] + 1)       ; maw_packagedata = [[0, maw_radius, maw_bot, strt, "SPECIFIED", nconn]]
i, j = maw_loc  ; obs_elev = {}   ; maw_conn = []   ; gwf_obs = []
for k in range(maw_lay[0], maw_lay[1] + 1, 1):       
    gwf_obs.append([f"H0_{k}", "HEAD", (k, i, j)])  ; maw_obs = [["H0", "HEAD", (0,)]]    ; iconn = 0  ; z = -5.0         # obs_dict_B = maw_obs

In [4]:
np.savetxt(output_folder / "01_maw_packagedata.csv", maw_packagedata, fmt ="%s", delimiter=",") 
np.savetxt(output_folder / "03_delr.csv", delr, delimiter=",") ;np.savetxt(output_folder / "04_delc.csv", delc, delimiter=",")    # delr delc

In [5]:
def calc_cond(area, l1, l2, k1, k2):
    c1 = area * k1 / l1 ; c2 = area * k2 / l2
    return c1 * c2 / (c1 + c2)
    
for k in range(maw_lay[0], maw_lay[1] + 1, 1):
    if k == maw_lay[0]:
        area = delc[i] * delr[j]   ; l1 = 2.5            ; l2 = 2.5                 ; cond = calc_cond(area, l1, l2, k33, maw_highK)
        maw_conn.append([0, iconn, k - 1, i, j, top, maw_bot, cond, -999.0])        ; tag = f"Q{iconn:02d}"       ; obs_elev[tag] = z   
        maw_obs.append([tag, "maw", (0,), (iconn,)])  ; gwf_obs.append([tag, "flow-ja-face", (k,i,j), (k-1,i,j)]) ; iconn += 1    ; z -= 2.5 # k+1
    area = delc[i] * 5.0           ; l1 = 0.5 * delr[j]  ; l2 = 0.5 * delr[j - 1]   ; cond = calc_cond(area, l1, l2, maw_highK, k11)
    maw_conn.append([0, iconn, k, i, j - 1, top, maw_bot, cond, -999.0])            ; tag = f"Q{iconn:02d}"       ; obs_elev[tag] = z
    maw_obs.append([tag, "maw", (0,), (iconn,)])      ; gwf_obs.append([tag, "flow-ja-face", (k,i,j), (k,i,j-1)]) ; iconn += 1               # j+1
    area = delr[j] * 5.0           ; l1 = 0.5 * delc[i]  ; l2 = 0.5 * delc[i - 1]   ; cond = calc_cond(area, l1, l2, maw_highK, k11)
    maw_conn.append([0, iconn, k, i - 1, j, top, maw_bot, cond, -999.0])            ; tag = f"Q{iconn:02d}"       ; obs_elev[tag] = z
    maw_obs.append([tag, "maw", (0,), (iconn,)])      ; gwf_obs.append([tag, "flow-ja-face", (k,i,j), (k,i-1,j)]) ; iconn += 1               # i+1        
    area = delc[i] * 5.0           ; l1 = 0.5 * delr[j]  ; l2 = 0.5 * delr[j + 1]   ; cond = calc_cond(area, l1, l2, maw_highK, k11)
    maw_conn.append([0, iconn, k, i, j + 1, top, maw_bot, cond, -999.0])            ; tag = f"Q{iconn:02d}"       ; obs_elev[tag] = z
    maw_obs.append([tag, "maw", (0,), (iconn,)])      ; gwf_obs.append([tag, "flow-ja-face", (k,i,j), (k,i,j+1)]) ; iconn += 1 ; z -= 5.0    # j+1 (2x)
    if k == maw_lay[1]:
        z += 2.5 ; area = delc[i] * delr[j] ; l1 = 2.5  ; l2 = 2.5                  ; cond = calc_cond(area, l1, l2, maw_highK, k33)
        maw_conn.append([0, iconn, k + 1, i, j, top, maw_bot, cond, -999.0])        ; tag = f"Q{iconn:02d}"       ; obs_elev[tag] = z
        maw_obs.append([tag, "maw", (0,), (iconn,)]) ; gwf_obs.append([tag, "flow-ja-face", (k, i, j), (k + 1, i, j)])     ; iconn += 1
        
obs_dict2 = maw_obs                                  ; ab=pd.DataFrame(obs_dict2) ; ab.to_csv(output_folder/"06_obs_dict.csv") 
np.savetxt(output_folder / "02_maw_conn.csv", maw_conn, delimiter=",") 

In [6]:
aux_ws = Path("./_aux") ; pth = '14'; fpth = os.path.join(aux_ws, pth, '14_reg_ex-gwf-maw-p03.hds')
h = flopy.utils.HeadFile(fpth).get_data()
f1 = 0.5 * (delr_r + delr[0]) / delc_r           ; f2 = 0.5 * (delr_r + delr[-1]) / delc_r
regional_range = [k for k in range(nlay_r)]      ; local_range = [[0]] + [[k, k + 1] for k in range(1, nlay, 2)]    ; chd_spd = []
for kr, kl in zip(regional_range, local_range):
    h1 = h[kr ,0 ,0]   ;h2=h[kr ,0 ,1]           ; hi1=h1+f1*(h2-h1);h1=h[kr ,0 ,2]             ; h2=h[kr ,0 ,3] ; hi2=h1+f2*(h2-h1)
    for k in kl:
        for il in range(nrow):     chd_spd.append([k, il, 0, hi1])                              ; chd_spd.append([k, il, ncol - 1, hi2])
chd_spd2 = pd.DataFrame(chd_spd) ; chd_spd2.to_csv (sim_ws/ "07_chd_spd.csv")                   ; print('regional heads', h.shape)         # !!

regional heads (21, 1, 200)


In [7]:
botm = np.arange(-5, aq_bottom - 5.0, -5.0)  ; icelltype2 = [1] + [0 for k in range(1, nlay)]   ; i, j = maw_loc
k11_sim = k11          ; k33_sim = k33   ; idomain = np.ones(shape3d, dtype=float)
for k in range(maw_lay[0], maw_lay[1] + 1, 1): idomain[k, i, j] = 0    ;sa=pd.DataFrame(idomain[0,:,:]);sa.to_csv(output_folder/"05_idomain.csv") 

In [8]:
obs_file = f"{sim_name}.maw.obs"      ; csv_file = obs_file +".csv"            ; obs_dict = { csv_file: maw_obs}       
head_filerecord = f"{sim_name}.hds"   ; budget_filerecord=f"{sim_name}.cbc"   
csv_file = obs_file+".csv"            ; obsdict = {csv_file:gwf_obs}                                            # err ?

sim = flopy.mf6.MFSimulation (sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6")
flopy.mf6.ModflowTdis (sim, nper=nper, perioddata=tdis_ds, time_units=time_units)
flopy.mf6.ModflowIms  (sim,print_option="summary",outer_maximum=nouter,outer_dvclose=hclose,
                       inner_maximum=ninner,inner_dvclose=hclose,rcloserecord=f"{rclose} strict")
gwf = flopy.mf6.ModflowGwf   (sim, modelname=sim_name, save_flows=True)
flopy.mf6.ModflowGwfdis   (gwf,length_units=length_units,nlay=nlay,nrow=nrow,ncol=ncol,delr=delr,delc=delc,top=top,botm=botm,idomain=idomain)
flopy.mf6.ModflowGwfnpf   (gwf,icelltype=icelltype2, k=k11_sim, k33=k33_sim, save_specific_discharge=True)
flopy.mf6.ModflowGwfic    (gwf, strt=strt)
flopy.mf6.ModflowGwfchd   (gwf, stress_period_data=chd_spd)
flopy.mf6.ModflowGwfrcha  (gwf, recharge=recharge)
maw = flopy.mf6.ModflowGwfmaw (gwf ,no_well_storage=True ,nmawwells=1   , packagedata = maw_packagedata    ,connectiondata=maw_conn)
maw.obs.initialize            (filename = obs_file ,digits = 10 ,print_input=True    , continuous  = obs_dict)
#flopy.mf6.ModflowUtlobs       (gwf ,filename = obs_file ,print_input=False    , continuous  = obsdict)           # err ?
flopy.mf6.ModflowGwfoc    (gwf ,head_filerecord=head_filerecord ,budget_filerecord=budget_filerecord ,
                           saverecord=[("HEAD" ,"LAST") ,("BUDGET" ,"LAST")] ,printrecord=[("BUDGET" ,"LAST")])
sim.write_simulation(silent=True)  ; sim.run_simulation(silent=True)

(True, [])

In [9]:
sim = flopy.mf6.MFSimulation.load(sim_ws = sim_ws ,exe_name = exe_name ,verbosity_level = 0) 
maw0 = pd.read_csv(sim_ws / 'ex-gwf-maw-p03.maw.obs.csv') ;maw = maw0.T; maw.head()

,0
time,1.000000
H0,4.925854
Q00,0.002057
Q01,0.427359
Q02,0.872791


In [10]:
gwf = sim.get_model(sim_name); extents = gwf.modelgrid.extent ; extents

(0.0, 99.99999999999999, 0.0, 50.0)